### Linear programming 
Linear programming is an optimization technique for a system of linear constraints and a linear objective function. An objective function defines the quantity to be optimized, and the goal of linear programming is to find the values of the variables that maximize or minimize the objective function.
Linear programming is useful for many problems that require an optimization of resources. It could be applied to manufacturing, to calculate how to assign labor and machinery to minimize cost of operations. It could be applied in high-level business operations, to decide which products to sell and in what quantity in order to maximize profit. It could also be applied in logistics, to decide how to apply resources to get a job done in the minimum amount of time.
Good introduction you can use here: https://brilliant.org/wiki/linear-programming/

Here we created one interesting task to illustrate LP problem.

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

### Task description
Imagine you are local community officer in new suburban municipality and you need to hire teachers for new schools. You are responsible for ten schools and you have asked the teachers where they live. You have read an article that people are more motivated for a job if they do not have to travell a long way, so you want to hire teachers in the school which is the nearest to their home. Also, teachers have some salary expectations and you have some budget constraints. Every school needs one new teacher. How would you solve this? By linear programming! But, let's create some fake data first.

In [2]:
# Let's create the list of the teachers first:
teachers = []
for i in range(1,101):
    teachers.append(f'teacher_{i}')

In [3]:
#then schools
schools = []
for i in range(1,11):
    schools.append(f'School_{i}')

In [4]:
#now we will for each teacher and school create some fake distance that in maximum could be 20 km
distances = []
for teacher in teachers:
    for school in schools:
        distances.append(np.random.randint(2, 20, size=1))
len(distances)   

1000

In [5]:
values = np.asarray(distances).reshape(100,-10)
df = pd.DataFrame(values, index=teachers, columns = schools)

In [6]:
df

,School_1,School_2,School_3,School_4,School_5,School_6,School_7,School_8,School_9,School_10
teacher_1,4,14,6,15,16,16,11,8,17,14
teacher_2,5,8,6,17,9,18,17,7,15,9
teacher_3,18,13,10,17,7,17,7,19,5,2
teacher_4,4,4,2,11,19,6,7,7,11,17
teacher_5,3,15,2,2,3,15,9,16,15,2
...,...,...,...,...,...,...,...,...,...,...
teacher_96,9,11,10,11,9,12,11,16,9,6
teacher_97,8,13,14,17,4,3,14,6,9,7
teacher_98,7,15,15,17,10,11,5,17,17,12
teacher_99,13,15,11,14,12,3,11,6,3,15


In [7]:
#At the end, let's add some fake salary expectaitons for each teacher, maximal teacher salary could be 8000 dollars
salary_expectation = []
for i in range(1,101):
    salary_expectation.append(np.random.randint(8000, size=1))
salary_expectation = np.asarray(salary_expectation).reshape(100,)

In [8]:
salary_pd = pd.Series(salary_expectation, index = df.index)

In [9]:
df['salary_expectation'] = salary_pd

In [10]:
#We must also calculate total distance for each teacher which we will use for objective function (we sum all school distances for each teacher)
df['total_distance'] = df.iloc[:,:-2].sum(axis=1)

In [11]:
df.head()

,School_1,School_2,School_3,School_4,School_5,School_6,School_7,School_8,School_9,School_10,salary_expectation,total_distance
teacher_1,4,14,6,15,16,16,11,8,17,14,5820,107
teacher_2,5,8,6,17,9,18,17,7,15,9,1251,102
teacher_3,18,13,10,17,7,17,7,19,5,2,1815,113
teacher_4,4,4,2,11,19,6,7,7,11,17,6253,71
teacher_5,3,15,2,2,3,15,9,16,15,2,4521,80


In [12]:
#Declare and initialise the model
#Resource Assignment problem - RAP
#Four components of the model: data, desicion variables, constraints and objective function

m = Model('RAP')

Restricted license - for non-production use only - expires 2023-10-25


In [13]:
from itertools import product

combinations = list(product(teachers, schools))
combinations[:10]

[('teacher_1', 'School_1'),
 ('teacher_1', 'School_2'),
 ('teacher_1', 'School_3'),
 ('teacher_1', 'School_4'),
 ('teacher_1', 'School_5'),
 ('teacher_1', 'School_6'),
 ('teacher_1', 'School_7'),
 ('teacher_1', 'School_8'),
 ('teacher_1', 'School_9'),
 ('teacher_1', 'School_10')]

In [14]:
# x is our decistion variable, in this case it could be only 1 (meaning this teacher goes to this school) or 0 (not assigned to this school)
# in combinations we created all possible combinations of each teacher and each school
x = m.addVars(combinations, name='assign')

In [15]:
#now we create constraint that teacher can be assigned only to one school
jobs = m.addConstrs((x.sum('*',school)==1 for school in schools), 'job')

In [16]:
# for all teachers in the list of teachers with index j must be less 1, i.e. some teachers could not be chosen
#teachers must be assigned maximum to 1 job, but it could happen teacher will not be chosen at all if he or she lives to far
# or have unrealistic salary expectation
resources = m.addConstrs((x.sum(teacher,'*')<=1 for teacher in teachers), 'teacher')

In [17]:
# just transform salary expectation to dict for gurobypi for budget constraint 
cost = {}
for i, teacher in enumerate(teachers):
    for j, school in enumerate(schools):
        cost[(teacher,school)] = df['salary_expectation'][i]

In [18]:
#We also have some budget constraint. Out total budget is 400 000 dolars. So salary expectations of selected teachers
#must not exceed 4000 000 dollars.
budget = 400000
budget = m.addConstr((x.prod(cost)<=budget), 'budget')

In [19]:
# now we transpose our pandas data frame in multidict gurobipy could use 
obj_func = {}
for i, teacher in enumerate(teachers):
    for j, school in enumerate(schools):
        obj_func[(teacher,school)] = df.iloc[i,j]

In [20]:
len(obj_func)

1000

In [21]:
obj_func

{('teacher_1', 'School_1'): 4,
 ('teacher_1', 'School_2'): 14,
 ('teacher_1', 'School_3'): 6,
 ('teacher_1', 'School_4'): 15,
 ('teacher_1', 'School_5'): 16,
 ('teacher_1', 'School_6'): 16,
 ('teacher_1', 'School_7'): 11,
 ('teacher_1', 'School_8'): 8,
 ('teacher_1', 'School_9'): 17,
 ('teacher_1', 'School_10'): 14,
 ('teacher_2', 'School_1'): 5,
 ('teacher_2', 'School_2'): 8,
 ('teacher_2', 'School_3'): 6,
 ('teacher_2', 'School_4'): 17,
 ('teacher_2', 'School_5'): 9,
 ('teacher_2', 'School_6'): 18,
 ('teacher_2', 'School_7'): 17,
 ('teacher_2', 'School_8'): 7,
 ('teacher_2', 'School_9'): 15,
 ('teacher_2', 'School_10'): 9,
 ('teacher_3', 'School_1'): 18,
 ('teacher_3', 'School_2'): 13,
 ('teacher_3', 'School_3'): 10,
 ('teacher_3', 'School_4'): 17,
 ('teacher_3', 'School_5'): 7,
 ('teacher_3', 'School_6'): 17,
 ('teacher_3', 'School_7'): 7,
 ('teacher_3', 'School_8'): 19,
 ('teacher_3', 'School_9'): 5,
 ('teacher_3', 'School_10'): 2,
 ('teacher_4', 'School_1'): 4,
 ('teacher_4', 'Sch

In [22]:
#Now we should define our objective function. It is a distance for each teacher from each school. We want to minimize it.
m.setObjective(x.prod(obj_func), GRB.MINIMIZE)

In [28]:
#We can save our model
# save model for inspection - you can see it in folder of this task.
m.write('TEACHERS.lp')

In [24]:
# run optimisation engine
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 111 rows, 1000 columns and 3000 nonzeros
Model fingerprint: 0xf363be96
Coefficient statistics:
  Matrix range     [1e+00, 8e+03]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+05]
Presolve time: 0.01s
Presolved: 111 rows, 1000 columns, 3000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.000000e+01   0.000000e+00      0s
      10    2.0000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.000000000e+01


In [25]:
#display optimal values of desicion variables:
#we print which teachers our solver selected for each school
for var in m.getVars():
    if var.x==1:
        print(var.varName, var.x)

assign[teacher_6,School_2] 1.0
assign[teacher_12,School_1] 1.0
assign[teacher_16,School_6] 1.0
assign[teacher_38,School_8] 1.0
assign[teacher_54,School_7] 1.0
assign[teacher_61,School_10] 1.0
assign[teacher_66,School_3] 1.0
assign[teacher_72,School_9] 1.0
assign[teacher_85,School_5] 1.0
assign[teacher_94,School_4] 1.0


In [26]:
#We basically solved the problem. Teacher 6 goes to school 2, teacher 12 to school 1, etc. 
#Maybe we could see values of our objective function:

In [27]:
#display optimal matching score
print('total optimised distance', m.ObjVal)
print('average distance of all teachers', df['total_distance'].mean())

total optimised distance 20.0
average distance of all teachers 92.86


It was easy for our solver to solve this. It is easy problem. Imagine now that we have 100 schools and 1000 teachers. The procedure would be completely the same.  